## decorator @

关于python的装饰器，在看infd的代码里有大量用到，说实话之前一直没去学，所以也有点难理解，做个记录。

In [7]:
def decorator(func):
    # func即为需要包装的函数
    def wrapper():
        # 内部嵌套一个包装函数，实际被执行的是wrapper
        print('before func excution')
        func()
        print('after func excution')
    return wrapper

@decorator
def say_hello():
    print('hello')

say_hello()
print(say_hello.__name__)


before func excution
hello
after func excution
wrapper


其实也很好理解，就是通过函数的嵌套给被修饰的函数套了一层衣服
但是观察say_hello.name的输出发现是wrapper，这是因为实际被调用的函数是wrapper
这一点导致原始函数的信息被修改，为了避免这一点，使用warps方法。

In [2]:
from functools import wraps

def decorator(func):
    @wraps(func)
    # 有点像是再套一层衣服
    def wrapper(*arg, **kwargs):
        print('before func excution')
        func(*arg, **kwargs)
        print('after func excution')
    return wrapper

@decorator
def add(num1, num2):
    print(num1 + num2)

# add = decorator(add)
add(1, 2)
print(add.__name__)

before func excution
3
after func excution
add


简单描述一下运行逻辑 

@decorator 会使得add = decorator(add)  （decorator返回wrapper）

本质上后面调用add已经变成了调用wrapper。

所以我作add（1，2）其实是在向wrapper传参

后面在wrapper里面参数再被传到add里

所以如果wrapper定义的括号里是空的 相当于向wrapper()传递1，2 这就会引发报错

---


用类装饰函数

In [10]:
# 正常的使用方式
class decorator:
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        print('before excution')
        result = self.func(*args, **kwargs)
        print('after excution')
        return result
    
@decorator
def add(a, b):
    return a + b

print(add(1, 2))

before excution
after excution
3


这种装饰方式下看不出任何异常 add的运行似乎和用函数装饰一致

事实并非如此 add装饰后已经变成了一个实例化的obbject

只是因为我们对decorator类的call方法的维护掩盖了这一点

下面更加明显地揭示这一点

In [ ]:
class decorator:
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwds):
        @wraps(self.func)
        def wrapper(*args, **kwargs):
            print('before excution')
            self.func(*args, **kwargs)
            print('after excution')
        return wrapper
    
@decorator
# add = decorator(add)
def add(a, b):
    print(a + b)

obb1 = add()
obb1(1, 2)

obb2 = add
obb2(1, 2)

before excution
3
after excution


<function __main__.add(a, b)>

@的运行逻辑还是一样 （如注释）

add()实际上是decorator类的实例化调用了call方法

add仅仅复制了一个实例

最后obb2的返回值没有正常输出 在jupyter的环境里似乎把它自动打印了